# Docker & SQL
In this homework we'll prepare the environment and practice with Docker and SQL

## Question 1. Knowing docker tags

Run the command to get information on Docker 

```docker --help```

Now run the command to get help on the "docker build" command:

```docker build --help```

Do the same for "docker run".

Which tag has the following text? - *Automatically remove the container when it exits* 

- `--delete`
- `--rc`
- `--rmc`
- `--rm`

```bash
docker run --help | grep "Automatically remove the container when it exits"
```

```bash
--rm
```

## Question 2. Understanding docker first run 

Run docker with the python:3.9 image in an interactive mode and the entrypoint of bash.
Now check the python modules that are installed ( use ```pip list``` ). 

What is version of the package *wheel* ?

- 0.42.0
- 1.0.0
- 23.0.1
- 58.1.0


```bash
docker run -i python:3.9 bash
pip list
```

```bash
Package    Version
---------- -------
wheel      0.42.0
```

# Prepare Postgres

Run Postgres and load data as shown in the videos
We'll use the green taxi trips from September 2019:

```wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz```

You will also need the dataset with zones:

```wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv```

Download this data and put it into Postgres (with jupyter notebooks or with a pipeline)

In [49]:
import duckdb

# Load csv files
taxi_zone_lookup = duckdb.read_csv("data/taxi_zone_lookup.csv")
green_tripdata = duckdb.read_csv("data/green_tripdata_2019-09.csv.gz")

## Question 3. Count records 

How many taxi trips were totally made on September 18th 2019?

Tip: started and finished on 2019-09-18. 

Remember that `lpep_pickup_datetime` and `lpep_dropoff_datetime` columns are in the format timestamp (date and hour+min+sec) and not in date.

- 15767
- 15612
- 15859
- 89009

In [50]:
duckdb.sql(
    """
    select 
        count(*) as taxi_trips
    from green_tripdata
    where datetrunc('day', lpep_pickup_datetime) = '2019-09-18'
    and datetrunc('day', lpep_dropoff_datetime) = '2019-09-18'
    limit 100
    """
)

┌────────────┐
│ taxi_trips │
│   int64    │
├────────────┤
│      15612 │
└────────────┘

## Question 4. Largest trip for each day

Which was the pick up day with the largest trip distance
Use the pick up time for your calculations.

- 2019-09-18
- 2019-09-16
- 2019-09-26
- 2019-09-21

In [51]:
duckdb.sql(
    """
    with 
    
    total_trip_distance as (
        select 
            datetrunc('day', lpep_pickup_datetime) as date,
            sum(trip_distance) as total_trip_distance, 
        from green_tripdata
        group by datetrunc('day', lpep_pickup_datetime)
        order by total_trip_distance desc
        limit 1
    ),

    longest_trip_distance as (
        select 
            datetrunc('day', lpep_pickup_datetime) as date,
            max(trip_distance) as longest_trip_distance, 
        from green_tripdata
        group by datetrunc('day', lpep_pickup_datetime)
        order by longest_trip_distance desc
        limit 1
    )

    select * from longest_trip_distance
    """
)

┌────────────┬───────────────────────┐
│    date    │ longest_trip_distance │
│    date    │        double         │
├────────────┼───────────────────────┤
│ 2019-09-26 │                341.64 │
└────────────┴───────────────────────┘

## Question 5. The number of passengers

Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?
 
- "Brooklyn" "Manhattan" "Queens"
- "Bronx" "Brooklyn" "Manhattan"
- "Bronx" "Manhattan" "Queens" 
- "Brooklyn" "Queens" "Staten Island"

In [63]:
duckdb.sql(
    """
    with green_tripdata_agg as (
        select
            PULocationID, 
            sum(total_amount) as total_amount
        from green_tripdata
        where datetrunc('day', lpep_pickup_datetime) = '2019-09-18'
        group by PULocationID

    ),

    taxi_zone_lookup_filtered as (
        select * 
        from taxi_zone_lookup
        where Borough != 'Unknown'
    ),

    joined as (
        select *
        from green_tripdata_agg
        inner join taxi_zone_lookup_filtered
        on (PULocationID = LocationID)
    ),

    answer as (
        select 
            Borough,
            sum(total_amount) as total_amount
        from joined
        group by Borough
        having sum(total_amount) > 50000
    )

    select * from answer
    
    """
)

┌───────────┬───────────────────┐
│  Borough  │   total_amount    │
│  varchar  │      double       │
├───────────┼───────────────────┤
│ Manhattan │ 92271.29999999967 │
│ Brooklyn  │ 96333.24000000008 │
│ Queens    │ 78671.71000000014 │
└───────────┴───────────────────┘

## Question 6. Largest tip

For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip?
We want the name of the zone, not the id.

Note: it's not a typo, it's `tip` , not `trip`

- Central Park
- Jamaica
- JFK Airport
- Long Island City/Queens Plaza

In [89]:
duckdb.sql(
    """
    with green_tripdata_reduced as (
        select
            PULocationID,
            DOLocationID,
            tip_amount
        from green_tripdata
        where strftime(lpep_pickup_datetime, '%Y-%m') = '2019-09'
    ),

    pickup_astoria as (
        select
            DOLocationID,
            tip_amount
        from green_tripdata_reduced
        join taxi_zone_lookup
        on (PULocationID = LocationID)
        where Zone = 'Astoria'
    ),

    answer as (
        select 
            Zone,
            max(tip_amount) as largest_tip
        from pickup_astoria
        join taxi_zone_lookup
        on (DOLocationID = LocationID)
        group by Zone
        order by largest_tip desc
        limit 1
    )

    select * from answer
    """
)

┌─────────────┬─────────────┐
│    Zone     │ largest_tip │
│   varchar   │   double    │
├─────────────┼─────────────┤
│ JFK Airport │       62.31 │
└─────────────┴─────────────┘